# Sensitivity Analysis

## 1. Define the problem and policies

In [4]:
# Imports

import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, MultiprocessingEvaluator, perform_experiments, ema_logging, Policy, Scenario)
ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, save_results, load_results
from specify import specify_levers


from SALib.analyze import sobol
from plotting_for_sobol import plot_sobol_indices, plot_scores


In [16]:
# Specification of the problem

from problem_formulation import get_model_for_problem_formulation
dike_model, planning_steps = get_model_for_problem_formulation(1)

#Specification of the policies

n_policies = 4
policy0 = Policy('policy_0', **{'0_RfR 0': 0, '0_RfR 1' : 0, '0_RfR 2' : 0,
                    '1_RfR 0': 0, '1_RfR 1' : 0, '1_RfR 2' : 0,
                    '2_RfR 0': 0, '2_RfR 1' : 0, '2_RfR 2' : 0,
                    '3_RfR 0': 0, '3_RfR 1' : 0, '3_RfR 2' : 0,
                    '4_RfR 0': 0, '4_RfR 1' : 0, '4_RfR 2' : 0,
                    'A.1_DikeIncrease 0' : 0, 'A.1_DikeIncrease 1' : 0, 'A.1_DikeIncrease 2' : 0,
                    'A.2_DikeIncrease 0' : 0, 'A.2_DikeIncrease 1' : 0, 'A.2_DikeIncrease 2' : 0,
                    'A.3_DikeIncrease 0' : 0, 'A.3_DikeIncrease 1' : 0, 'A.3_DikeIncrease 2' : 0,
                    'A.4_DikeIncrease 0' : 0, 'A.4_DikeIncrease 1' : 0, 'A.4_DikeIncrease 2' : 0,
                    'A.5_DikeIncrease 0' : 0, 'A.5_DikeIncrease 1' : 0, 'A.5_DikeIncrease 2' : 0,
                    'EWS_DaysToThreat':  0   }) # DO NOTHING
policy1 = Policy('policy_1', **{'0_RfR 0': 0, '0_RfR 1' : 0, '0_RfR 2' : 0,
                    '1_RfR 0': 0, '1_RfR 1' : 0, '1_RfR 2' : 0,
                    '2_RfR 0': 0, '2_RfR 1' : 0, '2_RfR 2' : 0,
                    '3_RfR 0': 0, '3_RfR 1' : 0, '3_RfR 2' : 0,
                    '4_RfR 0': 0, '4_RfR 1' : 0, '4_RfR 2' : 0,
                    'A.1_DikeIncrease 0' : 10, 'A.1_DikeIncrease 1' : 0, 'A.1_DikeIncrease 2' : 0,
                    'A.2_DikeIncrease 0' : 0, 'A.2_DikeIncrease 1' : 0, 'A.2_DikeIncrease 2' : 0,
                    'A.3_DikeIncrease 0' : 10, 'A.3_DikeIncrease 1' : 0, 'A.3_DikeIncrease 2' : 0,
                    'A.4_DikeIncrease 0' : 0, 'A.4_DikeIncrease 1' : 0, 'A.4_DikeIncrease 2' : 0,
                    'A.5_DikeIncrease 0' : 0, 'A.5_DikeIncrease 1' : 0, 'A.5_DikeIncrease 2' : 0,
                    'EWS_DaysToThreat':  0   }) # DO DIKES at A1, A3
policy2 = Policy('policy_2', **{'0_RfR 0': 0, '0_RfR 1' : 0, '0_RfR 2' : 0,
                    '1_RfR 0': 0, '1_RfR 1' : 0, '1_RfR 2' : 0,
                    '2_RfR 0': 0, '2_RfR 1' : 0, '2_RfR 2' : 0,
                    '3_RfR 0': 0, '3_RfR 1' : 0, '3_RfR 2' : 0,
                    '4_RfR 0': 0, '4_RfR 1' : 0, '4_RfR 2' : 0,
                    'A.1_DikeIncrease 0' : 10, 'A.1_DikeIncrease 1' : 0, 'A.1_DikeIncrease 2' : 0,
                    'A.2_DikeIncrease 0' : 10, 'A.2_DikeIncrease 1' : 0, 'A.2_DikeIncrease 2' : 0,
                    'A.3_DikeIncrease 0' : 10, 'A.3_DikeIncrease 1' : 0, 'A.3_DikeIncrease 2' : 0,
                    'A.4_DikeIncrease 0' : 0, 'A.4_DikeIncrease 1' : 0, 'A.4_DikeIncrease 2' : 0,
                    'A.5_DikeIncrease 0' : 10, 'A.5_DikeIncrease 1' : 0, 'A.5_DikeIncrease 2' : 0,
                    'EWS_DaysToThreat':  0   }) # DO DIKES at A1, A2, A3, A5
policy3 = Policy('policy_3', **{'0_RfR 0': 0, '0_RfR 1' : 0, '0_RfR 2' : 0,
                    '1_RfR 0': 1, '1_RfR 1' : 0, '1_RfR 2' : 0,
                    '2_RfR 0': 1, '2_RfR 1' : 0, '2_RfR 2' : 0,
                    '3_RfR 0': 1, '3_RfR 1' : 0, '3_RfR 2' : 0,
                    '4_RfR 0': 1, '4_RfR 1' : 0, '4_RfR 2' : 0,
                    'A.1_DikeIncrease 0' : 10, 'A.1_DikeIncrease 1' : 0, 'A.1_DikeIncrease 2' : 0,
                    'A.2_DikeIncrease 0' : 10, 'A.2_DikeIncrease 1' : 0, 'A.2_DikeIncrease 2' : 0,
                    'A.3_DikeIncrease 0' : 10, 'A.3_DikeIncrease 1' : 0, 'A.3_DikeIncrease 2' : 0,
                    'A.4_DikeIncrease 0' : 10, 'A.4_DikeIncrease 1' : 0, 'A.4_DikeIncrease 2' : 0,
                    'A.5_DikeIncrease 0' : 10, 'A.5_DikeIncrease 1' : 0, 'A.5_DikeIncrease 2' : 0,
                    'EWS_DaysToThreat':  4   }) # DO EVERYTHING

## 2. Sobol's scores

### 2.1 Execute Sensistivity Analysis on uncertainties

We will run a total of N(2K+2) = 5(2\*19+2) experiments, since the dike model has K=19 uncertainties and we use a baseline number of experiments equalling N=5

In [21]:
len(dike_model.uncertainties)

19

In [ ]:
# Convert the problem into an SA-equivalent problem

sa_problem = get_SALib_problem(dike_model.uncertainties)

In [24]:
# Execute Sensistivity Analysis over uncertainties

baseline_n_experiments_u = 5
start = time.time()

with MultiprocessingEvaluator(dike_model) as evaluator:
     experiments, outcomes = evaluator.perform_experiments(scenarios = baseline_n_experiments_u,
                                                           policies = [policy0, policy1, policy2, policy3], 
                                                           uncertainty_sampling = SOBOL)
end = time.time()
print('Sobol time is ' + str(round((end - start)/60)) + ' minutes')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 200 scenarios * 4 policies * 1 model(s) = 800 experiments
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 240 cases completed
[MainProcess/INFO] 320 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 480 cases completed
[MainProcess/INFO] 560 cases completed
[MainProcess/INFO] 640 cases completed
[MainProcess/INFO] 720 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Sobol time is 13 minutes


In [27]:
# save the results

u_results = (experiments, outcomes)
save_results(u_results, "../results/Sensitivity_"+str(baseline_n_experiments_u)+"ex_over_uncertainties_200s_4pol.tar.gz")

[MainProcess/INFO] results saved successfully to C:\Users\frac1\Documents\GitHub\Model Based DM - Assignment collaboration\model-based-decision-making\results\Sensitivity_5ex_over_uncertainties_200s_4pol.tar.gz


In [41]:
# load the results

experiments, outcomes = load_results("../results/Sensitivity_"+str(baseline_n_experiments_u)+"ex_over_uncertainties_200s_4pol.tar.gz")

[MainProcess/INFO] results loaded succesfully from C:\Users\frac1\Documents\GitHub\Model Based DM - Assignment collaboration\model-based-decision-making\results\Sensitivity_5ex_over_uncertainties_200s_4pol.tar.gz


### 2.2 Execute Sensistivity Analysis on levers

We will run a sensitivity analysis over a total of N(2K+2) = 1(2\*31+2) experiments, since the dike model has K=31 levers and we use a baseline number of experiments equalling N=1 under the reference scenario (set as the worst-case scenario, and found from the notebook 1 - Reference Case Scenario)

In [ ]:
len(dike_model.levers)

In [ ]:
# Convert the problem into an SA-equivalent problem

sa_problem = get_SALib_problem(dike_model.levers)

In [ ]:
# Load policy results of a NoAction policy under 500 scenarios (notebook 1-Reference Case Scenario)
[experiments, outcomes] = load_results("../results/500Scenarios_NoAction_PF1.tar.gz")
outcomes_df = pd.DataFrame(data=outcomes)

#Get index of worst-case scenario
index_wc = outcomes_df.sort_values("Expected Number of Deaths").tail(1).index
experiment_wc = experiments.iloc[index_wc]

#Set the reference scenario as the worst-case scenario
reference_scenarios = [Scenario(f"{index}", **row) for index, row in experiment_wc.iloc[0:,0:19].iterrows()]

In [ ]:
# Execute Sensistivity Analysis over levers

baseline_n_experiments_l = 1

start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
     experiments, outcomes = evaluator.perform_experiments(scenarios = reference_scenarios,
                                                           policies = baseline_n_experiments_l,
                                                           levers_sampling = SOBOL)
end = time.time()
print('Sobol time is ' + str(round((end - start)/60)) + ' minutes')

In [ ]:
#save the results

l_results = (experiments, outcomes)
save_results(l_results,  "../results/Sensitivity_"+str(baseline_n_experiments_l)+"ex_over_levers_RefScenario.tar.gz")

In [ ]:
#load the levers' results

#experiments, outcomes = load_results("../results/Sensitivity_"+str(baseline_n_experiments_l)+"ex_over_levers_RefScenario.tar.gz")

### 2.3 Calculate Sobol's scores

In [45]:
pd.DataFrame(experiments).shape[0] #tells the number of rows

(800, 53)

It will be done for uncertainties

In [51]:
N = baseline_n_experiments_u #baseline_n_experiments for uncertainties
K = len(dike_model.uncertainties) #number of uncertainties
sobol_multiplier = 2*K+2



policies = np.empty([len(experiments), 1])
k = 0
l = N*sobol_multiplier
for i in range (len(experiments)) : 
    policies[i] = k
    if (i == l - 1) : 
        k = k + 1
        l = l + n_scenarios * sobol_multiplier
policies = policies.astype(int)

data = pd.DataFrame.from_dict(outcomes)
data['Policy'] = policies

In [52]:
data['Policy']

0      0
1      0
2      0
3      0
4      0
      ..
795    1
796    1
797    1
798    1
799    1
Name: Policy, Length: 800, dtype: int32

In [ ]:
# Calculation of S1, S2, ST indices

problem = get_SALib_problem(dike_model.levers)
scores = []
for i in range(0, 1) : 
    scores.append(sobol.analyze(problem, outcomes_tup[0][:, i], calc_second_order=True))

In [ ]:
fig = plot_scores(scores[0], problem)
for i in range(1, 1) : 
    figi = plot_scores(scores[i], problem)
    figi.axes.append(fig)
plt.show()

In [ ]:
fig.set_size_inches(7,7)

figi = plot_sobol_indices(scores[0], problem, criterion='ST', threshold=0.005)
figi.set_size_inches(7,7)
figi.axes.append(fig)

plt.show()